1. Library yang digunakan


In [14]:
import os
import numpy as np
import librosa
import librosa.display
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers # type: ignore
from tensorflow.keras.layers import Dropout # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore
from keras.models import load_model # type: ignore
from skimage.transform import resize

2. Load dataset


In [15]:
def load_audio_files(root_path, n_mels=128, fixed_length=128):
    data = []
    labels = []
    for root, dirs, files in os.walk(root_path):
        for ayat_dir in dirs:
            ayat_path = os.path.join(root, ayat_dir)
            for file in os.listdir(ayat_path):
                if file.endswith(".mp3"):
                    file_path = os.path.join(ayat_path, file)
                    try:
                        # Memuat file audio
                        y, sr = librosa.load(file_path, sr=None)
                        # Mengonversi ke Mel-spectrogram
                        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
                        S_dB = librosa.power_to_db(S, ref=np.max)
                        # Menormalkan dimensi Mel-spectrogram ke fixed_length
                        if S_dB.shape[1] < fixed_length:
                            S_dB = np.pad(S_dB, ((0, 0), (0, fixed_length - S_dB.shape[1])), mode='constant')
                        elif S_dB.shape[1] > fixed_length:
                            S_dB = S_dB[:, :fixed_length]
                        data.append(S_dB)
                        labels.append(file)  # Gunakan nama file sebagai label
                    except Exception as e:
                        print(f"Error loading {file_path}: {e}")
    
    data = np.array(data)
    labels = np.array(labels)
    
    return data, labels

# Path ke direktori dataset Anda (Surah Al-Fatihah)
data_path = r'D:\Kuliah\S6\Studi Independen\project akhir\Bacaan Quran\Surah Al-Fatihah'

# Memuat dan mengonversi file audio ke Mel-spectrogram dengan dimensi tetap
data, labels = load_audio_files(data_path, n_mels=128, fixed_length=128)

# Debugging: Memastikan bentuk data sesuai dengan yang diharapkan
if len(data.shape) == 3:
    print(f"Loaded {data.shape[0]} audio files with shape {data.shape}")
else:
    print("No audio files were loaded correctly.")

# Menampilkan file yang berhasil dimuat
print("Files loaded:")
print(labels)

Loaded 798 audio files with shape (798, 128, 128)
Files loaded:
['1.mp3' '2.mp3' '3.mp3' '4.mp3' '5.mp3' '6.mp3' '7.mp3' '1.mp3' '2.mp3'
 '3.mp3' '4.mp3' '5.mp3' '6.mp3' '7.mp3' '1.mp3' '2.mp3' '3.mp3' '4.mp3'
 '5.mp3' '6.mp3' '7.mp3' '1.mp3' '2.mp3' '3.mp3' '4.mp3' '5.mp3' '6.mp3'
 '7.mp3' '1.mp3' '2.mp3' '3.mp3' '4.mp3' '5.mp3' '6.mp3' '7.mp3' '1.mp3'
 '2.mp3' '3.mp3' '4.mp3' '5.mp3' '6.mp3' '7.mp3' '1.mp3' '2.mp3' '3.mp3'
 '4.mp3' '5.mp3' '6.mp3' '7.mp3' '1.mp3' '2.mp3' '3.mp3' '4.mp3' '5.mp3'
 '6.mp3' '7.mp3' '1.mp3' '2.mp3' '3.mp3' '4.mp3' '5.mp3' '6.mp3' '7.mp3'
 '1.mp3' '2.mp3' '3.mp3' '4.mp3' '5.mp3' '6.mp3' '7.mp3' '1.mp3' '2.mp3'
 '3.mp3' '4.mp3' '5.mp3' '6.mp3' '7.mp3' '1.mp3' '2.mp3' '3.mp3' '4.mp3'
 '5.mp3' '6.mp3' '7.mp3' '1.mp3' '2.mp3' '3.mp3' '4.mp3' '5.mp3' '6.mp3'
 '7.mp3' '1.mp3' '2.mp3' '3.mp3' '4.mp3' '5.mp3' '6.mp3' '7.mp3' '1.mp3'
 '2.mp3' '3.mp3' '4.mp3' '5.mp3' '6.mp3' '7.mp3' '1.mp3' '2.mp3' '3.mp3'
 '4.mp3' '5.mp3' '6.mp3' '7.mp3' '1.mp3' '2.mp3' '3.mp3' '4.

3. Standarisasi Data

In [16]:
# Standarisasi data
scaler = StandardScaler()
data_flat = np.reshape(data, (data.shape[0], -1))
scaled_data = scaler.fit_transform(data_flat)

# Menampilkan statistik standarisasi
print("Statistik standarisasi:")
print(f"Mean setiap fitur: {scaled_data.mean(axis=0)}")
print(f"Deviasi standar setiap fitur: {scaled_data.std(axis=0)}")

Statistik standarisasi:
Mean setiap fitur: [-4.112571e-07 -2.879771e-07 -2.372235e-07 ... -3.686077e-08 -3.686077e-08
 -3.686077e-08]
Deviasi standar setiap fitur: [0.9999993  1.000001   1.0000007  ... 0.99999905 0.99999905 0.99999905]


4. Memisahkan Data Training dan Data Testing

In [17]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(data.shape, X_train.shape, X_test.shape,)

(798, 128, 128) (638, 128, 128) (160, 128, 128)


5. Membuat Data Latih Menggunakan Algoritma CNN + Membuat model evaluasi untuk mengukur tingkat akurasi

In [18]:
# Resize data ke (128, 128) dan reshape ke (n_samples, 128, 128, 1)
resized_X_train = np.array([resize(x, (128, 128)) for x in X_train])
resized_X_train = resized_X_train.reshape((resized_X_train.shape[0], 128, 128, 1))


# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Transform y_test
y_test_encoded = label_encoder.transform(y_test)

# Build the CNN model with Dropout and Regularization
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(len(np.unique(y_train_encoded)), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with early stopping
history = model.fit(X_train, y_train_encoded, epochs=20, validation_data=(X_test, y_test_encoded), callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test_encoded, verbose=2)
print(f'\nTest accuracy: {test_acc}')

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.1529 - loss: 47.4934 - val_accuracy: 0.1562 - val_loss: 5.8952
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 17s 796ms/step - accuracy: 0.3233 - loss: 5.7277 - val_accuracy: 0.5063 - val_loss: 5.0703
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 762ms/step - accuracy: 0.5388 - loss: 4.9727 - val_accuracy: 0.4125 - val_loss: 5.4929
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 19s 667ms/step - accuracy: 0.5806 - loss: 4.6865 - val_accuracy: 0.6500 - val_loss: 4.1700
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 673ms/step - accuracy: 0.6816 - loss: 4.0094 - val_accuracy: 0.6000 - val_loss: 4.0122
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 653ms/step - accuracy: 0.6406 - loss: 3.7648 - val_accuracy: 0.6375 - val_loss: 3.6664
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 629ms/step - accuracy: 0.7162 - loss: 3.5019 - val_accuracy: 0.7000 - val_loss: 3.3695
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 607ms/step - accuracy: 0.7614 - loss: 3.1946 - val_accura

In [19]:
# Menggunakan raw string untuk path dan format .keras yang disarankan
model.save(r'D:\Kuliah\S6\Studi Independen\project akhir\Bacaan Quran\Sahabat_Quran.keras')

Model Prediksi

In [21]:
# Fungsi untuk memuat file audio dan mengonversi ke Mel-spectrogram
def load_audio_files(root_path, n_mels=128, fixed_length=128):
    data = []
    labels = []
    for root, dirs, files in os.walk(root_path):
        for ayat_dir in dirs:
            ayat_path = os.path.join(root, ayat_dir)
            for file in os.listdir(ayat_path):
                if file.endswith(".mp3"):
                    file_path = os.path.join(ayat_path, file)
                    try:
                        y, sr = librosa.load(file_path, sr=None)
                        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
                        S_dB = librosa.power_to_db(S, ref=np.max)
                        if S_dB.shape[1] < fixed_length:
                            S_dB = np.pad(S_dB, ((0, 0), (0, fixed_length - S_dB.shape[1])), mode='constant')
                        elif S_dB.shape[1] > fixed_length:
                            S_dB = S_dB[:, :fixed_length]
                        data.append(S_dB)
                        labels.append(file)
                    except Exception as e:
                        print(f"Error loading {file_path}: {e}")
    
    data = np.array(data)
    labels = np.array(labels)
    
    return data, labels

# Path ke data baru untuk prediksi
new_data_path = r'D:\Kuliah\S6\Studi Independen\project akhir\aset lainnya\G'

# Memuat dan mengonversi file audio baru ke Mel-spectrogram
new_data, new_labels = load_audio_files(new_data_path, n_mels=128, fixed_length=128)
new_data = new_data.reshape((new_data.shape[0], 128, 128, 1))

# Memuat model yang telah dilatih
model = load_model(r'D:\Kuliah\S6\Studi Independen\project akhir\Bacaan Quran\Sahabat_Quran.keras', compile=False)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Definisikan fungsi prediksi di luar loop
@tf.function
def predict(model, data):
    return model(data)

# Melakukan prediksi
new_predictions = predict(model, new_data)
new_predictions = np.argmax(new_predictions, axis=1)

# Gunakan LabelEncoder yang sama
label_encoder = LabelEncoder()
label_encoder.fit(y_train)  # Pastikan ini sesuai dengan y_train yang digunakan selama pelatihan

# Transformasi prediksi dan true labels
new_predictions = label_encoder.inverse_transform(new_predictions)

# Pastikan true_labels sesuai dengan format yang diharapkan
true_labels = ['1.mp3', '2.mp3', '3.mp3', '4.mp3', '5.mp3', '6.mp3', '7.mp3']  # Sesuaikan dengan data Anda

for file, pred, true in zip(new_labels, new_predictions, true_labels):
    print(f"File: {file}, Predicted Ayat: {pred}, True Ayat: {true}")
    if pred == true:
        print("Prediksi benar")
    else:
        print("Prediksi salah")

File: 1.mp3, Predicted Ayat: 1.mp3, True Ayat: 1.mp3
Prediksi benar
File: 2.mp3, Predicted Ayat: 3.mp3, True Ayat: 2.mp3
Prediksi salah
File: 3.mp3, Predicted Ayat: 3.mp3, True Ayat: 3.mp3
Prediksi benar
File: 4.mp3, Predicted Ayat: 4.mp3, True Ayat: 4.mp3
Prediksi benar
File: 5.mp3, Predicted Ayat: 5.mp3, True Ayat: 5.mp3
Prediksi benar
File: 6.mp3, Predicted Ayat: 2.mp3, True Ayat: 6.mp3
Prediksi salah
File: 7.mp3, Predicted Ayat: 3.mp3, True Ayat: 7.mp3
Prediksi salah
